In [89]:
"""module docstring"""
import pandas as pd
from itertools import permutations
from entities import Bet, Competitor, Forecaster, Game
from utils import show

### an instance of this object for each round
class Round:
    def __init__(self, competitors, mode=1):
        self.competitors = [Competitor(name) for name in competitors]
        self.advance_to_next = [] # advance to next
        self.games_df = self.make_games_df(mode)
        #self.show_games_df = self.games_df.applymap(show)
  
    def make_players_df(self):
        '''represent the table of players'''
        dat = {c.name: [c.wins] for c in self.competitors}
        players = pd.DataFrame.from_dict(dat, orient='index', columns=['wins'])
        return players

    def make_games_df(self, mode=1): 
        '''return list of games for this round. 
           mode toggles each player plays once per game or the whole permutation set'''
        n = len(self.competitors)
        if mode=='exhaustive': 
            pairs = permutations(self.competitors, 2)
            games = [Game(ident=i, black=t[0], white=t[1]) for i,t in enumerate(pairs)]
        elif mode==1: 
            assert len(self.competitors)%2==0
            games = [Game(ident=k//2, black=self.competitors[k], white=self.competitors[k+1]) for k in range(0,n//2, 2)]
                
        games_dict = {g.ident: [g.black, g.white, g.winner] for g in games}                                                                                        
        games_df = pd.DataFrame.from_dict(games_dict, orient='index', columns=['black', 'white', 'winner'])
        
        return games_df
    
    def mark_winner(self, ident, victory): 
        self.games_df.iloc[ident].winner = self.games_df.iloc[ident][victory]
        pass
    
    def who_advances(self, by_name=False): 
        def who_advances_():
            self.advance_to_next = self.games_df.winner
            return self.advance_to_next
        if by_name: return who_advances_().apply(show).to_numpy()
        else: return who_advances_().to_numpy()

class Book(Round): 
    def __init__(self, forecasters):
        super()

In [90]:
one = Round([f'{c}' for c in "abcdefgh"])

one.mark_winner(1, 'white')
one.mark_winner(0, 'black')

one.games_df.applymap(show).isna().sum()

two = Round(one.who_advances(by_name=True))

two.games_df.applymap(show)

,black,white,winner
0,a,d,None


False